In [ ]:
import requests
import json
import time
import sys

from services.dte_tools.data_task_tools import (
    assert_dte_tools_available,
    get_resolved_parameters_for_connection,
    initialise_data_task,
    get_temp_dir,
    get_data_dir
)

In [ ]:
# Initialize the environment
params = {}
environment = initialise_data_task("DHIS2 SQL View Creation Task", params=params)
params["name"] = params.get("customname", params.get("name", "No parameters given!"))

if not params:
    environment.log_error("No parameters given!")

assert_dte_tools_available()

In [ ]:
# Resolve connection parameters
resolved_parameters = get_resolved_parameters_for_connection("DHIS2")
tmp_dir = get_temp_dir()

environment.log_message('SQL View Creation Starting!')
environment.log_message("Using parameters: %s" % resolved_parameters)
environment.log_message("Using temp directory: %s" % tmp_dir)

In [ ]:
# DHIS2 API credentials
DHIS2_BASE_URL = resolved_parameters["server"]
USERNAME = resolved_parameters["username"]
PASSWORD = resolved_parameters["password"]

In [ ]:
# API endpoints
DATASETS_ENDPOINT = f"{DHIS2_BASE_URL}/dataSets.json?fields=id,name&paging=false"
SQLVIEWS_ENDPOINT = f"{DHIS2_BASE_URL}/sqlViews.json"

In [ ]:
# Headers
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

In [ ]:
# Fetch all SQL views
def get_existing_sql_views():
    response = requests.get(SQLVIEWS_ENDPOINT, auth=(USERNAME, PASSWORD), headers=HEADERS)
    if response.status_code == 200:
        return {view["name"]: view["id"] for view in response.json().get("sqlViews", [])}
    elif response.status_code == 403:
        environment.log_error("User does not have permission to access SQL Views.")
        return None
    else:
        environment.log_error(f"Error fetching SQL views: {response.status_code}")
        environment.log_error(response.text)
        return None

In [ ]:
# Fetch all datasets
def get_datasets():
    response = requests.get(DATASETS_ENDPOINT, auth=(USERNAME, PASSWORD), headers=HEADERS)
    if response.status_code == 200:
        return response.json().get("dataSets", [])
    else:
        environment.log_error(f"Error fetching datasets: {response.status_code}")
        environment.log_error(response.text)
        return []

In [ ]:
# Construct and execute SQL query for each dataset
def execute_queries():
    existing_views = get_existing_sql_views()
    if existing_views is None:
        return
    
    datasets = get_datasets()
    for dataset in datasets:
        dataset_name = dataset["name"]
        view_name = f"Query for {dataset_name}"
        
        if view_name in existing_views:
            environment.log_message(f"Skipping {dataset_name}, SQL view '{view_name}' already exists.")
            continue
        
        environment.log_message(f"Creating SQL view for dataset: {dataset_name}")
        
        sql_query = f"""
        SELECT 
            dv.sourceid AS organisation_unit_id,
            ou.name AS organisation_unit_name,
            dv.dataelementid,
            de.name AS dataelement_name,
            dv.periodid,
            pe.startdate AS period_start,
            pe.enddate AS period_end,
            dv.value
        FROM datavalue dv
        JOIN datasetelement dse ON dv.dataelementid = dse.dataelementid
        JOIN dataset ds ON dse.datasetid = ds.datasetid
        JOIN dataelement de ON dv.dataelementid = de.dataelementid
        JOIN organisationunit ou ON dv.sourceid = ou.organisationunitid
        JOIN period pe ON dv.periodid = pe.periodid
        WHERE ds.name = '{dataset_name}';
        """
        
        payload = {
            "name": view_name,
            "description": f"Auto-generated SQL view for dataset: {dataset_name}",
            "cacheStrategy": "NO_CACHE",
            "type": "QUERY",
            "sqlQuery": sql_query
        }
        
        response = requests.post(
            SQLVIEWS_ENDPOINT,
            auth=(USERNAME, PASSWORD),
            headers=HEADERS,
            data=json.dumps(payload)
        )
        
        if response.status_code == 201:
            environment.log_message(f"SQL view '{view_name}' created successfully for dataset: {dataset_name}")
        elif response.status_code == 403:
            environment.log_error(f"Permission denied: Cannot create SQL view for {dataset_name}")
        else:
            environment.log_error(f"Error creating SQL view for {dataset_name}: {response.status_code}")
            environment.log_error(response.text)

In [ ]:
# Run the function to execute queries
execute_queries()

environment.log_message('SQL View Creation Completed!')
environment.log_message('Data upload completed!')